In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip uninstall moviepy decorator -y
!pip install moviepy

Found existing installation: moviepy 0.2.3.5
Uninstalling moviepy-0.2.3.5:
  Successfully uninstalled moviepy-0.2.3.5
Found existing installation: decorator 4.4.2
Uninstalling decorator-4.4.2:
  Successfully uninstalled decorator-4.4.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 388 kB 32.0 MB/s 
     |████████████████████████████████| 26.9 MB 1.2 MB/s 
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110742 sha256=1ab1201fc7c69a1e8653524f96c90ca4711e4fde9c597986d35f45fc8049cb88
  Stored in directory: /root/.cache/pip/wheels/56/dc/2b/9cd600d483c04af3353d66623056fc03faed76b7518faae4df
Successfully built moviepy
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.


In [ ]:
""" This file contains code for a fully convolutional
(i.e. contains zero fully connected layers) neural network
for detecting lanes. This version assumes the inputs
to be road images in the shape of 80 x 160 x 3 (RGB) with
the labels as 80 x 160 x 1 (just the G channel with a
re-drawn lane). Note that in order to view a returned image,
the predictions is later stacked with zero'ed R and B layers
and added back to the initial road image.
"""

import numpy as np
import pickle
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# Import necessary items from Keras
from keras.models import Sequential
from keras.layers import Activation, Dropout, UpSampling2D
from keras.layers import Conv2DTranspose, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
# from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers


def create_model(input_shape, pool_size):
    # Create the actual neural network here
    model = Sequential()
    # Normalizes incoming inputs. First layer needs the input shape to work
    model.add(BatchNormalization(input_shape=input_shape))

    # Below layers were re-named for easier reading of model summary; this not necessary
    # Conv Layer 1
    model.add(Conv2D(8, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv1'))

    # Conv Layer 2
    model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv2'))

    # Pooling 1
    model.add(MaxPooling2D(pool_size=pool_size))

    # Conv Layer 3
    model.add(Conv2D(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv3'))
    model.add(Dropout(0.2))

    # Conv Layer 4
    model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv4'))
    model.add(Dropout(0.2))

    # Conv Layer 5
    model.add(Conv2D(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv5'))
    model.add(Dropout(0.2))

    # Pooling 2
    model.add(MaxPooling2D(pool_size=pool_size))

    # Conv Layer 6
    model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv6'))
    model.add(Dropout(0.2))

    # Conv Layer 7
    model.add(Conv2D(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Conv7'))
    model.add(Dropout(0.2))

    # Pooling 3
    model.add(MaxPooling2D(pool_size=pool_size))

    # Upsample 1
    model.add(UpSampling2D(size=pool_size))

    # Deconv 1
    model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv1'))
    model.add(Dropout(0.2))

    # Deconv 2
    model.add(Conv2DTranspose(64, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv2'))
    model.add(Dropout(0.2))

    # Upsample 2
    model.add(UpSampling2D(size=pool_size))

    # Deconv 3
    model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv3'))
    model.add(Dropout(0.2))

    # Deconv 4
    model.add(Conv2DTranspose(32, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv4'))
    model.add(Dropout(0.2))

    # Deconv 5
    model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv5'))
    model.add(Dropout(0.2))

    # Upsample 3
    model.add(UpSampling2D(size=pool_size))

    # Deconv 6
    model.add(Conv2DTranspose(16, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Deconv6'))

    # Final layer - only including one channel so 1 filter
    model.add(Conv2DTranspose(1, (3, 3), padding='valid', strides=(1,1), activation = 'relu', name = 'Final'))

    return model

In [ ]:
# Load training images
train_images = pickle.load(open('/content/drive/My Drive/Colab Notebooks/Lane_Data/full_CNN_train.p', "rb" ))

# Load image labels
labels = pickle.load(open('/content/drive/My Drive/Colab Notebooks/Lane_Data/full_CNN_labels.p', "rb" ))

# Make into arrays as the neural network wants these
train_images = np.array(train_images)
labels = np.array(labels)

# Normalize labels - training images get normalized to start in the network
labels = labels / 255

# Shuffle images along with their labels, then split into training/validation sets
train_images, labels = shuffle(train_images, labels)
# Test size may be 10% or 20%
X_train, X_val, y_train, y_val = train_test_split(train_images, labels, test_size=0.1)

# Batch size, epochs and pool size below are all paramaters to fiddle with for optimization
batch_size = 128
epochs = 10
pool_size = (2, 2)
input_shape = X_train.shape[1:]

# Create the neural network
model = create_model(input_shape, pool_size)

# Using a generator to help the model use less data
# Channel shifts help with shadows slightly
datagen = ImageDataGenerator(channel_shift_range=0.2)
datagen.fit(X_train)

# Compiling and training the model
model.compile(optimizer='Adam', loss='mean_squared_error')
model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size), steps_per_epoch=len(X_train)/batch_size,
epochs=epochs, verbose=1, validation_data=(X_val, y_val))

# Freeze layers since training is done
model.trainable = False
model.compile(optimizer='Adam', loss='mean_squared_error')

# Save model architecture and weights
model.save('full_CNN_model.h5')

# Show summary of model
model.summary()

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


89/89 [==============================] - 25s 195ms/step - loss: 0.0517 - val_loss: 0.0220
Epoch 2/10
89/89 [==============================] - 14s 156ms/step - loss: 0.0161 - val_loss: 0.0139
Epoch 3/10
89/89 [==============================] - 14s 159ms/step - loss: 0.0123 - val_loss: 0.0106
Epoch 4/10
89/89 [==============================] - 14s 157ms/step - loss: 0.0104 - val_loss: 0.0098
Epoch 5/10
89/89 [==============================] - 14s 158ms/step - loss: 0.0095 - val_loss: 0.0091
Epoch 6/10
89/89 [==============================] - 14s 161ms/step - loss: 0.0089 - val_loss: 0.0091
Epoch 7/10
89/89 [==============================] - 14s 159ms/step - loss: 0.0085 - val_loss: 0.0082
Epoch 8/10
89/89 [==============================] - 14s 160ms/step - loss: 0.0082 - val_loss: 0.0083
Epoch 9/10
89/89 [==============================] - 14s 159ms/step - loss: 0.0076 - val_loss: 0.0068
Epoch 10/10
89/89 [==============================] - 14s 159ms/step - loss: 0.0070 - val_loss: 0.0067


In [ ]:
# Drawing Detected Lanes
import numpy as np
import cv2
import scipy
# from PIL import Image
# from scipy.misc import imresize
from moviepy.editor import VideoFileClip
from IPython.display import HTML
from keras.models import load_model


# Class to average lanes with
class Lanes():
    def __init__(self):
        self.recent_fit = []
        self.avg_fit = []


def road_lines(image):
    """ Takes in a road image, re-sizes for the model,
    predicts the lane to be drawn from the model in G color,
    recreates an RGB image of a lane and merges with the
    original road image.
    """
    # Get image ready for feeding into model
    small_img = cv2.resize(image,(160,80))
    small_img = np.array(small_img)
    small_img = small_img[None,:,:,:]

    # Make prediction with neural network (un-normalize value by multiplying by 255)
    prediction = model.predict(small_img)[0] * 255

    # Add lane prediction to list for averaging
    lanes.recent_fit.append(prediction)
    # Only using last five for average
    if len(lanes.recent_fit) > 5:
        lanes.recent_fit = lanes.recent_fit[1:]

    # Calculate average detection
    lanes.avg_fit = np.mean(np.array([i for i in lanes.recent_fit]), axis = 0)

    # Generate fake R & B color dimensions, stack with G
    blanks = np.zeros_like(lanes.avg_fit).astype(np.uint8)
    lane_drawn = np.dstack((blanks, lanes.avg_fit, blanks))

    # Re-size to match the original image
    lane_image = cv2.resize(lane_drawn, (1280, 720))
    # Merge the lane drawing onto the original image
    result = cv2.addWeighted(image, 1, lane_image, 1, 0, dtype = cv2.CV_32F)

    return result


In [ ]:
# Load Keras model
model = load_model('/content/full_CNN_model.h5')
# Create lanes object
lanes = Lanes()

# Where to save the output video
vid_output = 'proj_reg_vid.mp4'
# Location of the input video 
clip1 = VideoFileClip("/content/drive/My Drive/Colab Notebooks/Lane_Data/project_video.mp4")
# Create the clip
vid_clip = clip1.fl_image(road_lines)
# vid_clip = vid_clip.with_fps(24)
vid_clip.write_videofile(vid_output, audio=False)

1/1 [==============================] - 0s 425ms/step
Moviepy - Building video proj_reg_vid.mp4.
Moviepy - Writing video proj_reg_vid.mp4



t:   0%|          | 0/1260 [00:00<?, ?it/s, now=None]

1/1 [==============================] - 0s 16ms/step


t:   0%|          | 3/1260 [00:00<01:00, 20.87it/s, now=None]

1/1 [==============================] - 0s 16ms/step


...

t: 100%|█████████▉| 1259/1260 [02:43<00:00,  7.29it/s, now=None]

1/1 [==============================] - 0s 34ms/step


t: 100%|██████████| 1260/1260 [02:43<00:00,  6.96it/s, now=None]

1/1 [==============================] - 0s 27ms/step


Moviepy - Done !
Moviepy - video ready proj_reg_vid.mp4
